<a href="https://colab.research.google.com/github/kookeej/DILAB/blob/main/7.8-7.21/lab10_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lab10-2 - MNIST CNN
===
## 학습 단계
1. 라이브러리 가져오기 (torch, torchvision, matplotlib 등)
2. GPU 사용 설정하고 random value를 위한 seed 설정
3. 학습에 사용되는 parameter 설정(learning_rate, training_epochs, batch_size 등)
4. Dataset을 가져오고 학습에 쓰기 편리하도록 loader 만들기
5. 학습 모델 만들기(class CNN(torch.nn.Module)
6. Loss function(Criterion)을 선택하고 최적화 도구 선택(optimizer)
7. 모델 학습 및 loss check(Criterion의 output)
8. 학습된 모델의 성능을 확인한다.

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

* 먼저 필요한 라이브러리들을 import 해준다.

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)      # random value 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
    print('cuda is available!')

cuda is available!


* GPU를 사용할 것을 명시한다. random_value를 고정함으로써 예측가능한 난수를 발생시킨다.

In [3]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

* 학습에 필요한 parameter들을 설정한다.

In [4]:
# MNIST datasets
mnist_train = dsets.MNIST(root='MNIST_data/', train=True, 
                          transform=transforms.ToTensor(),
                          download=True)
mnist_test = dsets.MNIST(root='MNIST_data/', train=False, 
                         transform=transforms.ToTensor(),
                         download=True)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


* training set과 test set을 나눈다.

In [5]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [20]:
# CNN model 만들기
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()     #이 부분 빼먹으면 학습이 전혀 되지 않음
        self.layer1 = nn.Sequential(nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.MaxPool2d(2)
                                    )
        self.layer2 = nn.Sequential(
                                    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.MaxPool2d(2)
                                    )
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc1 = nn.Linear(3*3*128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)

        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return(out)

In [21]:
model = CNN().to(device)        # cuda로 사용한다.

value = torch.Tensor(1, 1, 28, 28).to(device)
print(model(value).shape)

torch.Size([1, 10])


* 모델을 설계한 후에는 반드시 모델이 제대로 설계가 되었는지 확인한다.

In [22]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [23]:
# training part
total_batch = len(data_loader)      # 전체 사이즈를 알 수 있다.

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()       # 이 부분 빼먹으면 학습 안됨. 매우 중요
        hypothesis = model(X)

        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch
    print('[Epoch:{}] cost={}'.format(epoch+1, avg_cost))

print('Learning finish!')

[Epoch:1] cost=0.15466617047786713
[Epoch:2] cost=0.04444815218448639
[Epoch:3] cost=0.029047904536128044
[Epoch:4] cost=0.021993059664964676
[Epoch:5] cost=0.017374075949192047
[Epoch:6] cost=0.014940028078854084
[Epoch:7] cost=0.012442213483154774
[Epoch:8] cost=0.011134816333651543
[Epoch:9] cost=0.009273744188249111
[Epoch:10] cost=0.008213501423597336
[Epoch:11] cost=0.008489972911775112
[Epoch:12] cost=0.008001129142940044
[Epoch:13] cost=0.004451865330338478
[Epoch:14] cost=0.007038424722850323
[Epoch:15] cost=0.005868325475603342
Learning finish!


In [26]:
# test part

with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1 ) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy', accuracy.item())

Accuracy 0.9833999872207642


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


* Layer를 더 깊게 쌓으면 더 좋은 성능을 보일까? 아니다. Layer를 깊게 쌓는 것보다는 효율성을 고려하며 설계하는 것이 더 중요하다.